In [3]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


Run on cluster

In [3]:
def create_cluster_script(inputs, iterations):
    
    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_analysis/patient_selection/confidence_intervals.sh'
    
    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(inputs)))
        out_file.write("#$ -l h_vmem=30G\n")
        out_file.write("#$ -tc 50\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set inputs=({0})\n".format(" ".join(inputs)))
        out_file.write("set iterations=({0})\n".format(" ".join(iterations)))
        out_file.write("\n")

        out_file.write("set input=$inputs[$SGE_TASK_ID]\n")
        out_file.write("set iteration=$iterations[$SGE_TASK_ID]\n")
        out_file.write("\n")
        
        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("\n")
  
        out_file.write("echo $input $iteration\n")
        out_file.write("\n")

        out_file.write("Rscript --vanilla /cellar/users/ramarty/Projects/hla_ii/data_analysis/patient_selection/prediction_with_confidence_intervals/confidence_intervals.R $input $iteration\n")
        out_file.write("\n")
                       
        out_file.write("date\n")

In [4]:
iterations = [str(x) for x in range(1, 1001)*3]
inputs = ['mhc_i' for i in range(1000)] + ['mhc_ii' for i in range(1000)] + ['both_classes' for i in range(1000)]

In [5]:
create_cluster_script(inputs, iterations)

Analyze output

In [ ]:
modelI, modelII, modelboth = [2, 2, 2]
threshold = 2
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions.mhc_i.model_{0}.{1}.10.data.txt'.format(modelI, 
                                                                                                                       threshold),
                 sep=' ').dropna()
precision_I, recall_I, thresholds = precision_recall_curve(df.label, df.predicted_prob, pos_label=1)
fpr_I, tpr_I, thresholds = metrics.roc_curve(df.label, df.predicted_prob, pos_label=1)
score_I = metrics.roc_auc_score(df.label, df.predicted_prob)
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions.mhc_ii.model_{0}.{1}.10.data.txt'.format(modelII, 
                                                                                                                        threshold), 
                 sep=' ').dropna()
precision_II, recall_II, thresholds = precision_recall_curve(df.label, df.predicted_prob, pos_label=1)
fpr_II, tpr_II, thresholds = metrics.roc_curve(df.label, df.predicted_prob, pos_label=1)
score_II = metrics.roc_auc_score(df.label, df.predicted_prob)
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions.both_classes.model_{0}.{1}.10.data.txt'.format(modelboth,
                                                                                                                              threshold),
                 sep=' ').dropna()
precision_both, recall_both, thresholds = precision_recall_curve(df.label, df.predicted_prob, pos_label=1)
fpr_both, tpr_both, thresholds = metrics.roc_curve(df.label, df.predicted_prob, pos_label=1)
score_both = metrics.roc_auc_score(df.label, df.predicted_prob)
sns.set_style("white")
plt.plot([0,1], [0, 1], c='k', label='Random')
plt.plot(fpr_II, tpr_II, c='g', label='MHC-II: {0}'.format(round(score_II, 2)))
plt.plot(fpr_I, tpr_I, c='b', label='MHC-I: {0}'.format(round(score_I, 2)))
plt.plot(fpr_both, tpr_both, c='r', label='MHC-II/MHC-I: {0}'.format(round(score_both, 2)))
plt.legend(loc=2, prop={'size': 6})
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.tight_layout()


In [25]:
scores, models, precisions = [], [], []
for model in ['mhc_i', 'mhc_ii', 'both_classes']:
    for iteration in range(1, 101):
        if iteration % 10 == 0:
            print iteration
        df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions/confidence_intervals/{0}/iteration_{1}.data.txt'.format(model, 
                                                                                                                        iteration),
                         sep=' ').dropna()
        score = metrics.roc_auc_score(df.label, df.predicted_prob)
        precision, recall, thresholds = metrics.precision_recall_curve(df.label, df.predicted_prob)
        scores.append(score)
        models.append(model)
        precisions.append(np.mean(precision))

10
20
30
40
50
60
70
80
90
100
10
20
30
40
50
60
70
80
90
100
10
20
30
40
50
60
70
80
90
100


In [26]:
results = pd.DataFrame({'Model': models,
                        'AUCs': scores,
                        'Precision': precisions})

In [27]:
results.groupby('Model').describe()

AUCs   Precision
Model                                     
both_classes count  100.000000  100.000000
             mean     0.618140    0.001906
             std      0.011572    0.000100
             min      0.593149    0.001674
             25%      0.611022    0.001835
             50%      0.619291    0.001905
             75%      0.624824    0.001980
             max      0.654485    0.002195
mhc_i        count  100.000000  100.000000
             mean     0.620391    0.001943
             std      0.009916    0.000095
             min      0.591162    0.001721
             25%      0.614293    0.001883
             50%      0.621563    0.001942
             75%      0.627265    0.001997
             max      0.648640    0.002235
mhc_ii       count  100.000000  100.000000
             mean     0.616523    0.001917
             std      0.011536    0.000093
             min      0.589995    0.001717
             25%      0.609137    0.001856
             50%      0.616362    0.001906
             75%      0.624695    0.001967
             max      0.652786    0.002236

In [20]:
precision, recall, thresholds = metrics.precision_recall_curve(df.label, df.predicted_prob)

In [24]:
np.median(precision)

0.0016367286016816004

Experimental

In [ ]:
# example from previous
sns.set_style("white")
fig = plt.figure(figsize=(3, 2.5))

modelI, modelII, modelboth = [2, 2, 2]
threshold = 2
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions.mhc_i.model_{0}.{1}.10.data.txt'.format(modelI, 
                                                                                                                       threshold),
                 sep=' ').dropna()
precision_I, recall_I, thresholds = precision_recall_curve(df.label, df.predicted_prob, pos_label=1)
fpr_I, tpr_I, thresholds = metrics.roc_curve(df.label, df.predicted_prob, pos_label=1)
score_I = metrics.roc_auc_score(df.label, df.predicted_prob)
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions.mhc_ii.model_{0}.{1}.10.data.txt'.format(modelII, 
                                                                                                                        threshold), 
                 sep=' ').dropna()
precision_II, recall_II, thresholds = precision_recall_curve(df.label, df.predicted_prob, pos_label=1)
fpr_II, tpr_II, thresholds = metrics.roc_curve(df.label, df.predicted_prob, pos_label=1)
score_II = metrics.roc_auc_score(df.label, df.predicted_prob)
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions.both_classes.model_{0}.{1}.10.data.txt'.format(modelboth,
                                                                                                                              threshold),
                 sep=' ').dropna()
precision_both, recall_both, thresholds = precision_recall_curve(df.label, df.predicted_prob, pos_label=1)
fpr_both, tpr_both, thresholds = metrics.roc_curve(df.label, df.predicted_prob, pos_label=1)
score_both = metrics.roc_auc_score(df.label, df.predicted_prob)
sns.set_style("white")
plt.plot([0,1], [0, 1], c='k', label='Random')
plt.plot(fpr_II, tpr_II, c='g', label='MHC-II: {0}'.format(round(score_II, 2)))
plt.plot(fpr_I, tpr_I, c='b', label='MHC-I: {0}'.format(round(score_I, 2)))
plt.plot(fpr_both, tpr_both, c='r', label='MHC-II/MHC-I: {0}'.format(round(score_both, 2)))
plt.legend(loc=2, prop={'size': 6})
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.tight_layout()
#plt.savefig('/cellar/users/ramarty/Data/hla_ii/generated_figures/predictions/Pan_AUC.pdf')

Trying to add confidence intervals

In [4]:
modelI, modelII, modelboth = [2, 2, 2]
threshold = 2

In [6]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions/predictions.mhc_i.model_{0}.{1}.10.data.txt'.format(modelI, 
                                                                                                                       threshold),
                 sep=' ').dropna()

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2868: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df.head()

,label,predicted,predicted_prob,label_fact
2280107,0.0,-6.469952,0.001549,0.0
4465664,0.0,-7.062235,0.000857,0.0
63674,0.0,-6.884375,0.001024,0.0
988334,0.0,-6.458738,0.001567,0.0
1042901,0.0,-6.479807,0.001534,0.0


In [8]:
len(df)

4697355

In [17]:
all_scores = []
for i in range(100):
    if i % 10 == 0:
        print i
    df_boot = df.sample(frac=1, replace=True)
    score_I = metrics.roc_auc_score(df_boot.label, df_boot.predicted_prob)
    all_scores.append(score_I)

0
10
20
30
40
50
60
70
80
90


In [23]:
print pd.Series(all_scores).quantile(0.025)
print pd.Series(all_scores).quantile(0.975)

0.570587088317
0.584019256625


In [24]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions/predictions.mhc_ii.model_{0}.{1}.10.data.txt'.format(modelI, 
                                                                                                                       threshold),
                 sep=' ').dropna()
all_scoresII = []
for i in range(100):
    if i % 10 == 0:
        print i
    df_boot = df.sample(frac=1, replace=True)
    score_I = metrics.roc_auc_score(df_boot.label, df_boot.predicted_prob)
    all_scoresII.append(score_I)

0
10
20
30
40
50
60
70
80
90


In [25]:
print pd.Series(all_scoresII).quantile(0.025)
print pd.Series(all_scoresII).quantile(0.975)

0.592599518945
0.605893406638


In [26]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/predictions/predictions.both_classes.model_{0}.{1}.10.data.txt'.format(modelI, 
                                                                                                                       threshold),
                 sep=' ').dropna()
all_scores_both = []
for i in range(100):
    if i % 10 == 0:
        print i
    df_boot = df.sample(frac=1, replace=True)
    score_I = metrics.roc_auc_score(df_boot.label, df_boot.predicted_prob)
    all_scores_both.append(score_I)

0
10
20
30
40
50
60
70
80
90


In [27]:
print pd.Series(all_scores_both).quantile(0.025)
print pd.Series(all_scores_both).quantile(0.975)

0.609456616844
0.622601954002
